线性规划调库流程总结：

- 1.导入数据，将数据用data.DataLoader打包成一个个batch
- 2.定义模型，这里使用nn.Linear，并将模型放到Sequential容器中，初始化模型参数
- 3.定义损失函数，这里使用MSELoss
- 4.定义优化器，这里使用SGD，并传入模型参数和学习率
- 5.训练模型，使用for循环，每次从dataloader中取出一个batch的数据，将其放入模型中，计算损失，反向传播，更新参数；每一次epoch，打印一次损失
- 6.评估模型

In [31]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [32]:
def test_star_symbol(*test):
    print(test)
    print(*test)

test_star_symbol(1,2,3,4,5)
test_star_symbol((1,2,3,4,5))
test_star_symbol(*(1,2,3,4,5)) # 与第一个等价

(1, 2, 3, 4, 5)
1 2 3 4 5
((1, 2, 3, 4, 5),)
(1, 2, 3, 4, 5)
(1, 2, 3, 4, 5)
1 2 3 4 5


In [33]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays) # 将输入的数据组合起来，*表示将传入的元组数据解包，解包后的数据为tensor，如果不解包，那么dataset中的数据为元组
    return data.DataLoader(dataset, batch_size, shuffle=is_train) # 返回一个数据迭代器，这个数据迭代器每次返回batch_size个数据，shuffle表示是否打乱数据，这里默认是打乱

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [34]:
next(iter(data_iter)) # 看看数据迭代器返回的数据是什么样子的

[tensor([[-0.5786,  0.2189],
         [ 0.0775, -0.8475],
         [ 1.9589,  1.2845],
         [ 0.2725,  0.2086],
         [ 0.4226,  0.6844],
         [ 0.0249, -0.3160],
         [ 1.2387, -0.6522],
         [-2.5800,  0.2143],
         [ 0.8455,  0.9739],
         [ 3.4283, -0.4749]]),
 tensor([[ 2.3031],
         [ 7.2429],
         [ 3.7529],
         [ 4.0312],
         [ 2.7092],
         [ 5.3418],
         [ 8.9024],
         [-1.6898],
         [ 2.5767],
         [12.6662]])]

In [35]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) # 线性回归模型，输入为2维，输出为1维，Sequential函数是将多个网络层组合起来，这里只有一个网络层

In [36]:
net[0].weight.data.normal_(0, 0.01) # 初始化权重，即w，这里用正态分布初始化，均值为0，标准差为0.01
net[0].bias.data.fill_(0) # 初始化偏差，即b

tensor([0.])

In [37]:
loss = nn.MSELoss() # 损失函数，这里用均方误差损失函数

In [38]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # 优化器，这里用梯度下降优化器，学习率为0.03，这里提前把网络的参数(w和b)传入优化器，这样优化器就知道要优化哪些参数了

In [39]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter: # 随机还是在这里，trainer只是更新参数
        l = loss(net(X) ,y) # 这里的net已经知道你当前的w和b了，所以这里的net(X)就是预测值，y就是真实值，然后计算损失
        trainer.zero_grad() # 梯度清零
        l.backward()
        trainer.step() # 更新参数
    # 这里没有no_grad，会被梯度追踪，但这里不需要计算梯度（backward），所以可以不加no_grad。
    # 不会影响下一个epoch的梯度计算，下一个epoch的l不是这里的l，不会从这里的l开始计算梯度。
    # 性能考虑，可以加上no_grad。
    l = loss(net(features), labels) 
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000168
epoch 2, loss 0.000097
epoch 3, loss 0.000096


In [40]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([1.3232e-05, 4.5705e-04])
b的估计误差： tensor([-0.0010])
